<a href="https://colab.research.google.com/github/deepak-raj007/Fake-news-classification-using-LSTM/blob/main/Fake_News_classification_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM,Embedding,Dense,Dropout
from sklearn.preprocessing import LabelEncoder

import re
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [3]:
news=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WELFake_Dataset.csv')
news.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [4]:
news['title'][101]

'For Helping Immigrants, Chobani’s Founder Draws Threats - The New York Times'

In [5]:
news['text'][0]

'No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called,  Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show, callers clearly call for  lynching  and  killing  of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be referred to

In [6]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [7]:
news.drop(columns=['Unnamed: 0'],inplace=True)

In [8]:
news_df=news.dropna()

In [9]:
news_df.shape

(71537, 3)

In [10]:
news_df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [11]:
X=news_df.drop('label',axis=1)

In [12]:
y=news_df['label']

In [13]:
X.shape

(71537, 2)

In [14]:
y.shape

(71537,)

In [15]:
#vocab size
voc_size=5000

Data preprocessing


In [16]:
msg=X.copy()

In [17]:
msg.reset_index(inplace=True)

In [18]:
msg['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [19]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(msg)):
    review = re.sub('[^a-zA-Z]', ' ', msg['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus[0]

'law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video'

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2477, 1105, 1288, 4036, 2044, 3160, 3276, 4101, 1903, 2984, 435, 4475],
 [3892,
  2547,
  3197,
  2764,
  1539,
  2760,
  3902,
  4209,
  4927,
  4441,
  3596,
  3625,
  4499,
  4475],
 [2612, 3012, 3959, 232, 829, 4890, 3753, 3931, 4332, 4331, 777, 502],
 [3359, 3546, 4709, 2731, 3834, 702, 1474, 4834, 1986, 3515, 1550],
 [4186, 3753, 717, 3881, 2251, 4338, 3788, 2658, 717],
 [4180, 2322, 1350, 3352, 1535, 1988, 2965, 1200, 3472, 186, 2150],
 [3962, 2542, 407, 556, 3546, 349, 4890, 577, 4148, 4475],
 [4510,
  2955,
  2707,
  805,
  2129,
  2383,
  327,
  1860,
  4088,
  4510,
  1797,
  487,
  3100,
  4250,
  4475],
 [3921, 2540, 3562, 2129, 2397, 138, 3622, 2540],
 [2342, 373, 1207, 2757, 437, 3515, 3027, 1404],
 [474, 3578, 879, 2665, 3523, 3750, 2391, 211, 2391, 2813],
 [2476, 2384, 556, 2308, 481, 2200, 2302, 1570, 3077],
 [4655, 2520, 4694, 2384, 370, 4297, 2658, 4185, 728, 1817, 2585],
 [2684, 4710, 3541, 3213, 3642, 3945, 2279, 326, 1605, 2010],
 [4714, 455, 4021, 586, 4267, 3

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2984  435 4475]
 [   0    0    0 ... 3625 4499 4475]
 [   0    0    0 ... 4331  777  502]
 ...
 [   0    0    0 ... 4170 4478 4044]
 [   0    0    0 ...  793 3006 3737]
 [   0    0    0 ... 2640 4185 2444]]


In [23]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape


(71537, (71537,))

In [25]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape


((71537, 20), (71537,))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [28]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
749/749 [==============================] - 42s 51ms/step - loss: 0.3322 - accuracy: 0.8470 - val_loss: 0.2611 - val_accuracy: 0.8868
Epoch 2/10
749/749 [==============================] - 37s 49ms/step - loss: 0.2270 - accuracy: 0.9055 - val_loss: 0.2552 - val_accuracy: 0.8919
Epoch 3/10
749/749 [==============================] - 42s 56ms/step - loss: 0.1918 - accuracy: 0.9222 - val_loss: 0.2721 - val_accuracy: 0.8886
Epoch 4/10
749/749 [==============================] - 36s 48ms/step - loss: 0.1648 - accuracy: 0.9352 - val_loss: 0.2768 - val_accuracy: 0.8897
Epoch 5/10
749/749 [==============================] - 38s 51ms/step - loss: 0.1410 - accuracy: 0.9450 - val_loss: 0.2979 - val_accuracy: 0.8881
Epoch 6/10
749/749 [==============================] - 36s 48ms/step - loss: 0.1184 - accuracy: 0.9535 - val_loss: 0.3315 - val_accuracy: 0.8878
Epoch 7/10
749/749 [==============================] - 43s 57ms/step - loss: 0.0982 - accuracy: 0.9621 - val_loss: 0.3764 - val_accuracy:

In [29]:
predictions = (model.predict(X_test)>0.5).astype("int32")



738/738 [==============================] - 6s 8ms/step


In [30]:
from sklearn.metrics import confusion_matrix


In [31]:
confusion_matrix(predictions,y_test)


array([[ 9857,  1008],
       [ 1836, 10907]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(predictions,y_test)

0.8795323619112165

Adding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])